In [1]:
import pandas as pd 
import numpy as np 
import os
import time
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

2.3.1


### 一、导入数据

In [2]:
train_file='./train.csv'
eval_file='./eval.csv'

In [3]:
#导入训练集和测试集
train_df=pd.read_csv(train_file)
eval_df=pd.read_csv(eval_file)

In [4]:
#pop函数可以将dataFrame中的某个字段提出掉,同时指定某个变量去接受去掉的那个字段
y_train=train_df.pop('survived')
y_eval=eval_df.pop('survived')

### 二、将csv的数据包装成tensorflow可以识别的的格式

In [5]:
#离散特征群
categorical_columns=['sex','n_siblings_spouses','parch','class','deck',
    'embark_town','alone']
#连续特征群
numeric_columns=['age','fare']

feature_columns=[]

for categorical_column in categorical_columns:
    vocab=train_df[categorical_column].unique()
    print(categorical_column,vocab)
    #transform categorical type feature
    #feature and feature`s value
    vocab_list=tf.feature_column.categorical_column_with_vocabulary_list(
        categorical_column,vocab)
    vocab_list=tf.feature_column.indicator_column(vocab_list)
    #append categorical data to feature_columns()
    feature_columns.append(vocab_list)

for categorical_column in numeric_columns:
    #transform numeric type feature
    #must define data type to float
    num_list=tf.feature_column.numeric_column(categorical_column,dtype=tf.float32)
    #append numeric data to feature_columns()
    feature_columns.append(num_list)

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [6]:
feature_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town',

### 三、定义创建dataset数据集函数

In [7]:
def make_dataset(data_df,label_df,epochs=10,shuffle=True,batch_size=32):
    #must transform dataFrame type to key:value type
    dataset=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
        dataset=dataset.shuffle(10000)
    dataset=dataset.repeat(epochs).batch(batch_size)
    return dataset

### 四、创建训练和测试样本

In [8]:
#创建train and test 的dataset
train_dataset=make_dataset(train_df,y_train,epochs=100,)
eval_dataset=make_dataset(eval_df,y_eval,epochs=1,shuffle=False)

### 五、创建神经网络模型

In [9]:
#用keras建立神经网络模型
model=keras.models.Sequential()
#first-tier must DenseFeatures()
model.add(keras.layers.DenseFeatures(feature_columns))
'''
    use the module keras.layers.DenseFeature
    if values are categorical type then execute one-hot transform
    if values are numeric type then keep the original data content unchanged
'''
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(2,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
                optimizer=keras.optimizers.SGD(lr=0.01),
             metrics=['accuracy'])

### 六、模型训练

In [10]:
#训练模型
history=model.fit(train_dataset,
                  validation_data=eval_dataset,
                 steps_per_epoch=20,
                 validation_steps=8,
                 epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
 1/20 [>.............................] - ETA: 0s - loss: 2.5634 - accuracy: 0.3125WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'sex': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'n_siblings_spouses': <tf.Tensor 'ExpandDims_